# Data Prep+Aug

In [1]:
import pandas as pd
import numpy as np

import os
import csv
import cv2

In [2]:
driving_log = pd.read_csv('sample_data/driving_log.csv')
print('Number of examples: {}'.format(driving_log.shape[0]))

Number of examples: 8036


In [3]:
# initial loop, no augmentation

"""
for line in lines[1:]:
    fn = line[0] #.split('/')[-1]
    img = cv2.imread('sample_data/'+fn)
    images.append(img)
    measurements.append(float(line[3]))    
"""
print()

In [4]:
# second loopo, added flipping

"""
for line in lines[1:]:
    fn = line[0] #.split('/')[-1]
    img = cv2.imread('sample_data/'+fn)
    
    images.append(img)
    measurements.append(float(line[3]))    
        
    images.append(cv2.flip(img,1))
    measurements.append( -1.0 * float(line[3])) 
"""
print()

In [5]:
# third loop, added multiple cameras
"""
CORRECTION = 0.2

images = []
measurements = []

with open('sample_data/driving_log.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader, None)
    c = 0
    for row in reader:
        img_center = cv2.imread('sample_data/' + row[0].strip())
        img_left = cv2.imread('sample_data/' + row[1].strip())
        img_right = cv2.imread('sample_data/' + row[2].strip())
        img = [img_center, img_left, img_right]
        
        steer_center = float(row[3])
        steer_left = steer_center + CORRECTION
        steer_right = steer_center - CORRECTION
        steer = [steer_center, steer_left, steer_right]

        # normal
        images.extend(img)
        measurements.extend(steer)    

        # flipped
        images.extend([cv2.flip(i,1) for i in img])
        measurements.extend([-1.0 * s for s in steer])   
        
        if c > 5000: break
        c += 1
        
X_train = np.array(images)
y_train = np.array(measurements)

"""
print()

In [15]:
# fourth attempt, using a generator

PATH = 'sample_data/'
CORRECTION = 0.2

def get_batch(data, size=1024):
    
    n = data.shape[0]
    
    for i in range(0, n, size):
        images = []
        measurements = []        
        j = min(n, i + size)
        
        # print(i, ' - ', j-1)
        
        for _,row in data[i:j].iterrows():
            img_center = cv2.imread(PATH + row[0].strip())
            img_left = cv2.imread(PATH + row[1].strip())
            img_right = cv2.imread(PATH + row[2].strip())
            img = [img_center, img_left, img_right]

            steer_center = float(row[3])
            steer_left = steer_center + CORRECTION
            steer_right = steer_center - CORRECTION
            steer = [steer_center, steer_left, steer_right]

            # normal
            images.extend(img)
            measurements.extend(steer)    

            # flipped
            images.extend([cv2.flip(i,1) for i in img])
            measurements.extend([-1.0 * s for s in steer])  
            

        X = np.array(images)
        y = np.array(measurements)

        yield X,y

---
# Model Architecture

In [16]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers import Cropping2D

#from keras.layers.pooling

In [17]:
model = Sequential()

# normalize
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320, 3)))

# crop
model.add(Cropping2D(cropping=((70,25), (0,0))))

In [18]:
# dummy architecture
"""
model.add(Flatten())
model.add(Dense(1))
"""

'\nmodel.add(Flatten())\nmodel.add(Dense(1))\n'

In [19]:
# nvidia architecture

model.add(Convolution2D(nb_filter=24,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(nb_filter=36,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(nb_filter=48,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))

model.add(Convolution2D(nb_filter=64,  nb_row=3, nb_col=3, activation='relu'))
model.add(Convolution2D(nb_filter=64,  nb_row=3, nb_col=3, activation='relu'))

model.add(Flatten())

model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(1))

In [20]:
model.compile(loss='mse', optimizer='adam')

---

# Training

In [26]:
EPOCHS = 2
BATCH_SIZE = 1024

In [27]:
# full mode

# model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=EPOCHS, verbose=1)

In [ ]:
# batch mode

for e in range(EPOCHS+1):
    print('*****************************************************************')
    print('Pass {}/{}'.format(e+1, EPOCHS))
    source = get_batch(driving_log, BATCH_SIZE)
    
    for X_train, y_train in source:
        model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=1)#, verbose=1)
    # X_train, y_train = next(source)
    

*****************************************************************
Pass 1/2
Train on 4915 samples, validate on 1229 samples
Epoch 1/2
4915/4915 [==============================] - 9s - loss: 0.0093 - val_loss: 0.0198
Epoch 2/2
4915/4915 [==============================] - 9s - loss: 0.0077 - val_loss: 0.0188
Train on 4915 samples, validate on 1229 samples
Epoch 1/2
4915/4915 [==============================] - 9s - loss: 0.0117 - val_loss: 0.0104
Epoch 2/2
4915/4915 [==============================] - 9s - loss: 0.0096 - val_loss: 0.0114
Train on 4915 samples, validate on 1229 samples
Epoch 1/2
4915/4915 [==============================] - 9s - loss: 0.0079 - val_loss: 0.0105
Epoch 2/2
4915/4915 [==============================] - 9s - loss: 0.0052 - val_loss: 0.0103
Train on 4915 samples, validate on 1229 samples
Epoch 1/2
4915/4915 [==============================] - 9s - loss: 0.0199 - val_loss: 0.0314
Epoch 2/2
4915/4915 [==============================] - 9s - loss: 0.0152 - val_loss: 0.03

In [ ]:
model.save('models/momo4.h5')